
## P7: Résolvez des problèmes en utilisant des algorithmes en Python



### references used:


[(c) Analyse de la complexité des algorithmes — Wikipédia](https://fr.wikipedia.org/wiki/Analyse_de_la_complexit%C3%A9_des_algorithmes)



[Parsing CSV Files with Python's DictReader](https://brodan.biz/blog/parsing-csv-files-with-python/)

I had an interview today (spoiler: I didn't get an offer), and one of the rounds of my interview involved refactoring some poorly written [Python](https://www.python.org/) code.

[TimeComplexity - Python Wiki](https://wiki.python.org/moin/TimeComplexity)

This page documents the time-complexity (aka "Big O" or "Big Oh") of various operations in current CPython. Other Python implementations (or older or still-under development versions of CPython) may have slightly different performance characteristics. However, it is generally safe to assume that they are not slower by more than a factor of O(log n).

[GitHub - pberkes/big_O: Python module to estimate big-O time complexity from execution time](https://github.com/pberkes/big_O)

big_O is a Python module to estimate the time complexity of Python code from its execution time. It can be used to analyze how functions scale with inputs of increasing size.







[Comment trier un dictionnaire Python par valeur | Delft Stack](https://www.delftstack.com/fr/howto/python/how-to-sort-a-dictionary-by-value/)

```python
sortedDict = sorted(exampleDict.items(), key=lambda x: x[1])
#Out: [('fourth', 1), ('third', 2), ('first', 3), ('second', 4)]
```

`exempleDict.items()` retourne une liste de paires clé-valeur du dictionnaire et le type de données de son élément est tuple. `x` est l’élément de ce tuple, où `x[0]` est la clé et `x[1]` est la valeur. `key=lambda x:x[1]` indique que la clé de comparaison est la valeur des éléments du dictionnaire.

In [1]:
import csv as csv
import re as re
import time
#from functools import wraps


In [2]:
try:
    import big_o
except ModuleNotFoundError:
    print('cherche',ModuleNotFoundError)
    !pip install --upgrade pip
    !pip install big_o
    import big_o

In [3]:
# constant
FILE = "data/p7-20-shares.csv" 
FIELDNAMES = ['name', 'cost', 'profit_share', 'profit']
PRECISION = 3
BUDGET = 50000

### En complément de Big_O, mesure du temps

Si la mesure du temps d'execution est dépendante du type de PC, des processus en cours d'execution etc., elle permet une approximation de l'efficacité  des algorithme choisis complémentaire à celle de la notation big_o.

Suivant le conseil de [Marina Mele](http://www.marinamele.com/author/marina-melegmail-com) dans [7 tips to Time Python scripts and control Memory & CPU usage](https://www.marinamele.com/7-tips-to-time-python-scripts-and-control-memory-and-cpu-usage)


In [4]:
 def fn_timer(function):
        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        print ("Total time running %s: %s nanoseconds" %
               (function.__name__, str(t1-t0))
               )
        return result
    return function_timer

In [5]:
# nettoyer les données des fichiers en entrée
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre

### Amélioration des données en V2

1. Sur python le travail avec float est plus couteux qu'en entier.
Une solution est alors de multiplier par 100 cout, budget et profit car cela ne change pas
le résultat.

2. Le tri du meilleur profit à contrainte de budget, dans la cas de l'algorithme glouton qui privilégie un tri au meilleur profit des actions peut evt. être amélioré en considérant le meilleur profit pour un cout de budget constant ; càd tri par profit/cout.

In [6]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
try:
    with open(FILE, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=';', doublequote=False)
        # skip the header
        next(csv_reader)
        for idx, line in enumerate(csv_reader):
            clean_data = True
            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False
            if line[FIELDNAMES[1]] != "":
                cout = 100 * int(clean_char(line[FIELDNAMES[1]]))
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
            if line[FIELDNAMES[3]] != "":
                gain = int(100 * float(clean_char(line[FIELDNAMES[3]])))
            else:
                print(f" line {idx} had missing profit data; dropped.")
                clean_data = False
            if (gain < 0) or (cout < 0):
                print(f" line {idx} had negative value; accepted but pls check.")
            if (clean_data):
                action_dict[cle] = (cout, gain)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")            


In [7]:
action_dict
                

{'Action-1': (2000, 100),
 'Action-2': (3000, 300),
 'Action-3': (5000, 750),
 'Action-4': (7000, 1400),
 'Action-5': (6000, 1019),
 'Action-6': (8000, 2000),
 'Action-7': (2200, 154),
 'Action-8': (2600, 286),
 'Action-9': (4800, 624),
 'Action-10': (3400, 918),
 'Action-11': (4200, 714),
 'Action-12': (11000, 990),
 'Action-13': (3800, 874),
 'Action-14': (1400, 14),
 'Action-15': (1800, 54),
 'Action-16': (800, 64),
 'Action-17': (400, 48),
 'Action-18': (1000, 140),
 'Action-19': (2400, 504),
 'Action-20': (11400, 2052)}

## 1. Algorithme Glouton

[Algorithme glouton — Wikipédia](https://fr.wikipedia.org/wiki/Algorithme_glouton)

Un **algorithme glouton** (_greedy algorithm_ en anglais, parfois appelé aussi algorithme gourmand, ou goulu) est un [algorithme](https://fr.wikipedia.org/wiki/Algorithmique "Algorithmique") qui suit le principe de faire, étape par étape, un choix optimum local, dans l'espoir d'obtenir un résultat optimum global. Par exemple, dans le problème du rendu de monnaie (donner une somme avec le moins possible de pièces), l'algorithme consistant à répéter le choix de la pièce de plus grande valeur qui ne dépasse pas la somme restante est un algorithme glouton.
Dans le système de pièces (1, 3, 4), l'algorithme glouton n'est pas optimal, comme le montre l'exemple simple suivant. Il donne pour 6 : 4+1+1, alors que 3+3 est optimal. 
Mais optimal par rapport à quelle contrainte? à priori le nombre pièce utilisé. Si c'était avoir moins de pièce dans son portemonnaie, la 1ère solution était "optimale".

### 1.2 Amélioration gloutonne



In [8]:
@fn_timer
def algo_glouton(dictio: dict[str, tuple], budget: float) -> (float, list):
    """trions la liste par le plus grand profit avant d'épuiser le budget"""
    action_trie = dict(sorted(dictio.items(), 
                              key=lambda x: x[1][1], reverse=True))

    budget_left = budget
    action_selected: list[str] = []
    for cle, valeur in action_trie.items():
        if budget_left - valeur[0] > 0:
            action_selected.append(cle)
            budget_left -= valeur[0]
    return (budget_left, action_selected)


In [9]:
# executons la fonction sur les données précédentes
reste_budget, actions_choisies = algo_glouton(action_dict,BUDGET)
total_profit: int = round(sum(list(map(lambda x: action_dict[x][1],actions_choisies))),2)
print('At a cost of ',BUDGET-reste_budget,actions_choisies,'brought ', total_profit, ' profit.')

Total time running algo_glouton: 22000 nanoseconds
At a cost of  49600 ['Action-20', 'Action-6', 'Action-4', 'Action-5', 'Action-12', 'Action-10', 'Action-19', 'Action-17'] brought  8931  profit.


In [10]:
# Parfait, ce n'est peut-être pas l'optimum mais c'est mieux que précédement.
actions_restantes = {cle:valeur for cle,valeur in action_dict.items() if cle not in actions_choisies}
actions_restantes

{'Action-1': (2000, 100),
 'Action-2': (3000, 300),
 'Action-3': (5000, 750),
 'Action-7': (2200, 154),
 'Action-8': (2600, 286),
 'Action-9': (4800, 624),
 'Action-11': (4200, 714),
 'Action-13': (3800, 874),
 'Action-14': (1400, 14),
 'Action-15': (1800, 54),
 'Action-16': (800, 64),
 'Action-18': (1000, 140)}

#### Version 2

In [11]:
def algo_glouton2(dictio: dict[str,tuple],budget: int) -> (int, list):
    """ Tri par densité de profit càd que les actions au meilleur rapport profit/cout
        sont les premières dans l'ordre
    """
    action_trie = dict(sorted(dictio.items(), key=lambda x:x[1][1]/x[1][0], reverse=True))
    budget_left = budget
    action_selected: list[str] = []
    for cle,valeur in action_trie.items():
        if budget_left - valeur[0] > 0:
            action_selected.append(cle)
            budget_left -= valeur[0]
    return (budget_left,action_selected)

    

In [12]:
# V2 : executons la fonction sur les données précédentes
reste_budgetv2, actions_choisiesv2 = algo_glouton(action_dict,BUDGET)


Total time running algo_glouton: 18200 nanoseconds


In [13]:
print(actions_choisiesv2)
print(actions_choisies)

['Action-20', 'Action-6', 'Action-4', 'Action-5', 'Action-12', 'Action-10', 'Action-19', 'Action-17']
['Action-20', 'Action-6', 'Action-4', 'Action-5', 'Action-12', 'Action-10', 'Action-19', 'Action-17']


In [14]:
total_profitv2: float = round(sum(list(map(lambda x: action_dict[x][1],actions_choisiesv2))),2)
print('At a cost of ',BUDGET-reste_budgetv2,actions_choisies,'brought ', total_profitv2, ' profit.')

At a cost of  49600 ['Action-20', 'Action-6', 'Action-4', 'Action-5', 'Action-12', 'Action-10', 'Action-19', 'Action-17'] brought  8931  profit.


* dans notre cas de figure, le tri par densité n'apporte pas de changement dans le profit réalisé.
cout = 496 et profit = 89.31.


## 2. Force brute ou explosion combinatoire

[Recherche exhaustive — Wikipédia](https://fr.wikipedia.org/wiki/Recherche_exhaustive)

Le principe de cet algorithme est d'essayer toutes les possibilités dans un intervalle. Un exemple courant est l'attaque par force brute des mots de passe. Si on sait que le mot de passe contient obligatoirement 4 chiffres, alors on essaie tous les nombres de 0000 à 9999 jusqu'à trouver le bon mot de passe.

Il s'agit de calculer toutes les combinaisons d'action de ce portefeuille, d'en évaluer le cout et le profit puis de retenir la meilleure combinaison = le plus haut profit à hauteur du budget.

Il faut un algorithme capable de calculer toutes les combinaisons de k pris par n actions, ici n = 20 et k = 1 à 20.

$
{n \choose k}={\frac {n!}{k!(n-k)!}}.
{20 \choose k}={\frac {20!}{k!(20-k)!}}.
$

### 2.1 Estimation simple du nombre de calcul.

In [15]:
def number_combi(n: int, k: int) -> int:
    def factorielle(x:int) -> int:
        if x <=1:
            return 1
        else:
            return (x * factorielle(x-1))
    top = factorielle(n)
    bot = factorielle(k) * factorielle(n-k)
    if bot > 0:
        return top/bot
    else:
        raise Error

In [16]:
N = len(action_dict.keys())
number_of_combi = 0
for K in range(N):
    number_of_combi += number_combi(N, K)
print(number_of_combi)   

1048575.0



Nous sommes prévenu, il y a environ 1 million de combinaison d'un portefeuille de 20 actions!
Le calcul des combinaisons va être long!!  
  
### 2.2 Comment déterminer toutes les combinaisons possibles?  
  
* Comment lister toutes les combinaisons de r objets parmi n ?  
[énumération des combinaisons](http://villemin.gerard.free.fr/Denombre/CbinEnum.htm)  
  
**Attention à l'explosion combinatoire!**  
  
##### 2.2.1 brute de brut  
  
Pour les combinaisons de k dans une liste de n éléments, on parcourt la liste, à partir du 1élément que l'on fixe et on cherche toutes les combinaisons de k-1 car des n-1 éléments restants.  
Exemple court :  
liste [1,2,3,4] N=len(liste) disons K=2 alors les 6 combinaisons seront:  
1,2 ; 1,3 ; 1,4;  
2,3 ; 2,4;  
3,4  


#### 2.2.2 Fonction combinatoire proposée.

In [17]:
# détermination itérative des combinaisons possibles de longueur length
# utilisation yield au lieu de return pour une meilleure efficacité en mémoire

def choose_iter(elements, length, track=False):
    if track: print('iteration loop on ',elements)
    # pour le 1er élément de la combinaison, il peut prendre toute valeur de la liste
    for i in range(len(elements)):
        # singelton -> on rapporte des t-uple de 1 avec le 1er élément lui-même
        if track: print('next first: [',elements[i], '] at position: ',i)
        if length == 1:
            yield [elements[i],]
        else:
            # on prend la liste réduite à partir de l'élément suivant i et 
            # on en cherche les combinaisons de longueur -1
            for next1 in choose_iter(elements[i+1:len(elements)], length-1):
                # et on retourne la liste précédent  plus la liste de l'appel suivant
                if track: print('    next t-uple,',next1, ' at position: ',i,' of sub-list:',elements[i+1:len(elements)])
                yield [elements[i],] + next1
# toutes les combinaisons de k éléments de la liste l

def choose(l, k, track=False):
    return list(choose_iter(l, k, track))

#### 2.2.3 Jouons avec la fonction pour en comprendre le fonctionnement

In [18]:
# pour mémoire nombre d'élément demandé, nbr min, nombre max de chiffre de l'élément
#def integers(n, min_, max_):
#    """ Return sequence of N random integers between min_ and max_ (included).
#    """
#    return [random.randint(min_, max_) for _ in range(n)]


list_2 = big_o.datagen.integers(4, 1, 10)


In [19]:
choose(list_2,3, track=True)

iteration loop on  [3, 4, 5, 5]
next first: [ 3 ] at position:  0
    next t-uple, [4, 5]  at position:  0  of sub-list: [4, 5, 5]
    next t-uple, [4, 5]  at position:  0  of sub-list: [4, 5, 5]
    next t-uple, [5, 5]  at position:  0  of sub-list: [4, 5, 5]
next first: [ 4 ] at position:  1
    next t-uple, [5, 5]  at position:  1  of sub-list: [5, 5]
next first: [ 5 ] at position:  2
next first: [ 5 ] at position:  3


[[3, 4, 5], [3, 4, 5], [3, 5, 5], [4, 5, 5]]

#### 2.2.4 Utilisation pour générer les combinaisons de la Force Brute

In [20]:
liste_action_pf = [x[0] for x in action_dict.items()]


In [21]:
@fn_timer
def algo_force_brute(portefeuille: list) -> list:
    liste_combinaison = []
    for bn_element_tuple in range(len(liste_action_pf)+1):
        liste_combinaison.extend(choose(liste_action_pf, bn_element_tuple))
    return liste_combinaison

In [22]:
combinaison_force_brute = algo_force_brute(liste_action_pf)
print(combinaison_force_brute[10:13])

Total time running algo_force_brute: 11223486900 nanoseconds
[['Action-11'], ['Action-12'], ['Action-13']]


**A l'impression, message d'averstissement Jupyter:**

```
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)
```
Par contre, l'output s'affiche partiellement là ou le print bloque.

In [23]:
#len(combinaison_force_brute)
#>1048575

In [24]:
# comptage par taille de tuple & long 
if False:
    dict_combinaison_taille = {x:[] for x in range(1,len(liste_action_pf)+1)}
# comptage par taille de tuple selon la taille de l'élément
    for el in map(lambda x:(len(x),x) ,combinaison_force_brute):
        dict_combinaison_taille[el[0]].append(el[1])
    for niveau in range(1,len(dict_combinaison_taille)+1):
        print(niveau,len(dict_combinaison_taille[niveau]))

##### comptage par taille de tuple selon la taille de l'élément  
1 20  
2 190  
3 1140  
4 4845  
5 15504  
6 38760  
7 77520  
8 125970  
9 167960  
10 184756  
11 167960  
12 125970  
13 77520  
14 38760  
15 15504  
16 4845  
17 1140  
18 190  
19 20  
20 1  
  
## 2.3 Explosion Force brute!  
Les éléments sont en place ; on connait le million de combinaison d'action possible ; calculons en cout et profit et déterminons le meilleur.  
Allons y!  

Struture des données à combiner : précédement nous avons passé une liste en paramètre à la fonction choose pour construire les combinaisons.
Sachant que la construction utilise le parcours de liste (for in range() ), la liste semble adaptée pour la construction des combinaisons.
Liste(combinaisons, cout, profit) : une liste de tuple ; combinaisons étant une liste d'action.

Structure du programme en 3 étapes :
1. construire toutes les combinaisons
2. évaluer cout et profit de chaque combinaison
3. détermination de la meilleure solution


In [25]:
# en entrée : liste de liste de str = liste de liste d'action
combinaison_force_brute[30000:30005]

[['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-12', 'Action-19'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-12', 'Action-20'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-14'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-15'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-16']]

In [26]:
@fn_timer
def calcul_combinaison(liste_act: list[str]) -> (list, int, int):
    """A chaque combinaison associer cout et profit
        """
    liste_valorisee = []
    for share in liste_act:
        cout_combinaison = sum( (map(lambda x: action_dict[x][0], share)))
        profit_combinaison = sum( (map(lambda x: action_dict[x][1], share)))
        liste_valorisee.append((share, cout_combinaison, profit_combinaison))
    return liste_valorisee

In [27]:
calcul_combinaison([['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-12', 'Action-19'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-12', 'Action-20'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-14'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-15'],
 ['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-16']])

Total time running calcul_combinaison: 19300 nanoseconds


[(['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-12', 'Action-19'],
  29000,
  4245),
 (['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-12', 'Action-20'],
  38000,
  5793),
 (['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-14'],
  20800,
  3639),
 (['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-15'],
  21200,
  3679),
 (['Action-1', 'Action-5', 'Action-10', 'Action-11', 'Action-13', 'Action-16'],
  20200,
  3689)]

In [29]:
# calcul complet
combinaisons_valorisees = calcul_combinaison(combinaison_force_brute)

Total time running calcul_combinaison: 4470688800 nanoseconds


Résultat 

Total time running calcul_combinaison: 4495526300 nanoseconds

[(['Action-1'], 2000, 100),  
 (['Action-2'], 3000, 300),  
 (['Action-3'], 5000, 750),  
 (['Action-4'], 7000, 1400),  
 (['Action-5'], 6000, 1019),  
 (['Action-6'], 8000, 2000),  
 
 
 #### Etape 3 déterminer l'élément à plus haut profit pour un budget <= 50000

In [ ]:
resultat = max(combinaisons_valorisees)